In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['CRYPTOGRAPHY_OPENSSL_NO_LEGACY'] = '1'
os.environ['HF_DATASETS_OFFLINE'] = '1'
os.environ['TRANSFORMERS_OFFLINE'] = '1'

import datasets
from datasets import load_dataset, Dataset, DatasetDict, Dataset
from tqdm.auto import tqdm
import json

datasets.disable_caching()

/mnt/nas/home/genggui/miniconda3/envs/megatron-next/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from megatron.energon import get_train_dataset, WorkerConfig, get_loader, DefaultTaskEncoder, Cooker, basic_sample_keys
from megatron.energon import DefaultTaskEncoder, TextSample, Sample
import gzip
import pickle
import webdataset as wds
import json
import dataclasses
import torch

from tqdm.auto import tqdm
from typing import List

from qwen3_base_energon_helpers import MyTaskEncoder, print_error_handler

In [3]:
tokenizer_path="/mnt/nas/home/genggui/code/Megatron-Next/model_dir/pulse_v18_3_32b_gemini_bf16/tokenizer_v18_gemini"

In [4]:
worker_config = WorkerConfig(
    rank=0,
    world_size=1,
    seed_offset=123456789,
    num_workers=1,
)

train_ds = get_train_dataset(
    '/mnt/nas/home/genggui/code/Megatron-Next/model_dir/pulse_v18_3_32b_gemini_bf16/dataset.yaml',
    split_part="train",
    task_encoder=MyTaskEncoder(
        tokenizer_path=tokenizer_path,
        seq_length=66536,
        sensitive_words_path=None,
    ),
    batch_size=1,
    packing_buffer_size=None,
    shuffle_buffer_size=None,
    max_samples_per_sequence=None,
    worker_config=worker_config,
)

train_dataloader = get_loader(train_ds, worker_config=worker_config,)

rank=0, worker=0: sample_range=[0, 98994] in 1 slices, sum(count)=98994: indexes=[0, 98994] slices=[train-chunk-0.tar[0(start), 98994(end)]]
rank=0, worker=0: sample_range=[0, 33580] in 1 slices, sum(count)=33580: indexes=[0, 33580] slices=[train-chunk-0.tar[0(start), 33580(end)]]
rank=0, worker=0: sample_range=[0, 70519] in 1 slices, sum(count)=70519: indexes=[0, 70519] slices=[train-chunk-0.tar[0(start), 70519(end)]]
rank=0, worker=0: sample_range=[0, 99256] in 1 slices, sum(count)=99256: indexes=[0, 99256] slices=[train-chunk-0.tar[0(start), 99256(end)]]
rank=0, worker=0: sample_range=[0, 14160] in 1 slices, sum(count)=14160: indexes=[0, 14160] slices=[train-chunk-0.tar[0(start), 14160(end)]]
rank=0, worker=0: sample_range=[0, 5231] in 1 slices, sum(count)=5231: indexes=[0, 5231] slices=[train-chunk-0.tar[0(start), 5231(end)]]
rank=0, worker=0: sample_range=[0, 1453] in 1 slices, sum(count)=1453: indexes=[0, 1453] slices=[train-chunk-0.tar[0(start), 1453(end)]]
rank=0, worker=0: sam

/mnt/nas/home/genggui/miniconda3/envs/megatron-next/lib/python3.12/site-packages/megatron/energon/loader.py:108: FutureWarning: Passing a worker_config to get_loader() is deprecated and will have no effect.
  warn_deprecated(


In [5]:
dd = iter(train_dataloader)

Token indices sequence length is longer than the specified maximum sequence length for this model (67720 > 66536). Running this sequence through the model will result in indexing errors


In [6]:
import json

count_map = {}

with open("/mnt/nas/home/genggui/code/Megatron-Next/model_dir/pulse_v18_3_32b_gemini_bf16/qua/pulse_v18_3_32b_gemini_train.jsonl", "w", encoding="utf8") as f:
    for _ in tqdm(range(2048)):
        item = next(dd)
        
        kk = item['__key__'][0].split("/")[-1].split("-train-")[0]
        if kk not in count_map:
            count_map[kk] = 0
        
        count_map[kk] += 1
              
        item = {
            "input_ids": item['input_ids'].tolist()[0],
            "labels": item['labels'].tolist()[0],
        }
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
        

            
        

100%|██████████| 2048/2048 [00:18<00:00, 108.53it/s]


In [7]:
# count_list = sorted(count_map.items(), key=lambda x:-x[1])
# [(k,v/2048)for k,v in count_list]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [6]:
tokenizer("<|im_start|>assistant\n<think>\n").input_ids 

[151644, 77091, 198, 151667, 198]

In [10]:
item = next(dd)
# print(item['__key__'][0].split("/")[-1].split("-train-")[0])
print(tokenizer.decode([item for item in item['input_ids'].tolist()[0] if item != -100]))

<|im_start|>user
是否正确，即如果一个 field extension $K/F$ 的 Galois group 是 isomorphic 于 $S_3$，那么它对应于 splitting field 的 某个 irreducible cubic polynomial 在 $F[x]$ 中？<|im_end|>
<|im_start|>assistant
<think>
以下是一种可能导致关于$S_3$伽罗瓦群和三次多项式的详细解释的思考过程：

1.  **分析用户的问题：**
    *   用户提问：“如果域扩张$K/F$的伽罗瓦群同构于$S_3$，那么它是否对应于$F[x]$中某个不可约三次多项式的分裂域？是否正确？”
    *   关键词：域扩张$K/F$，伽罗瓦群，同构于$S_3$，分裂域，不可约三次多项式，$F[x]$。
    *   核心问题是：$Gal(K/F) \cong S_3$是否*蕴含*这样的多项式存在？

2.  **初步反应与高层次回答：**
    *   是的，这个陈述是正确的。这是伽罗瓦理论中的一个经典结果。
    *   $S_3$的结构是关键。它是三个元素的置换群。伽罗瓦群置换多项式的根。一个不可约三次多项式有三个根。$S_3$是三个根置换的最大可能群。因此，这感觉非常合理。
    *   回应计划应为：
        a.  明确陈述该陈述是正确的。
        b.  提供一个证明或逻辑论证。
        c.  给出一个具体例子（例如$\mathbb{Q}$上的$x^3 - 2$）。
        d.  考虑逆命题：一个不可约三次多项式的分裂域是否总是具有伽罗瓦群$S_3$？这增加了深度并展示了更全面的理解。
        e.  总结关键要点。

3.  **充实证明/论证（第b部分）：**
    *   从假设开始：设$K/F$是一个伽罗瓦扩张，且$Gal(K/F) \cong S_3$。
    *   我们对$S_3$了解什么？
        *   阶为$3! = 6$。因此，$[K:F] = 6$。
        *   它有子群。它们是什么？
            *   恒等元{e}（阶为1）
            *   三个阶为2的子群（同构于$S_2

In [12]:
item['cu_seqlens']

tensor([    0,  4972,  5910,  6932,  8037, 46443, 47327, 47960, 49813, 51716,
        51846, 53869, 58505, 59626, 59816, 60048, 60807, 61753, 61886, 61909,
        62344, 65206, 66259, 66358, 66498, 66528], dtype=torch.int32)